In [8]:
import requests as r
import pandas as pd
import geopandas as gpd

In [9]:
# these are production keys
api_key = 'cvgH5DGPxjVJ3Az7mUNvuL2pPm8opLbA'
api_key_secret = 'AGYFOP7bP9zzC5xv'

In [10]:
url = 'https://api.amadeus.com/v1/security/oauth2/token'
header = {'Content-Type': 'application/x-www-form-urlencoded'}
body = {'grant_type' : 'client_credentials',
        'client_id': api_key,
        'client_secret' : api_key_secret}
resp = r.post(url,headers=header,data=body)
access_token = resp.json()['access_token']
access_token

'cxR0N7bertAauetekQN9NmhUCCWA'

In [18]:
header = {'Authorization' : f'Bearer {access_token}'}
params = {'origin' : 'CHI', # IATA code of the city from which the flight will depart
         'departureDate' : '2023-01-26,2023-05-31', #,2022-08-06 # format of YYYY-MM-DD format, e.g. 2017-12-25. Ranges are specified with a comma and are inclusive. Must be within 180 days
         'oneWay' : False, # boolean
         'duration' : 7, #the amount of days must be greater than 1 and less than 15
         'nonStop' : True } # boolean
resp = r.get('https://api.amadeus.com/v1/shopping/flight-destinations', headers=header,params=params).json()

data = []
for x in resp['data']:
    x['price'] = x['price']['total']
    data.append(x)

df = pd.DataFrame(resp['data'])
af = pd.read_csv('airport-codes-cleaned.csv')

def iata_to_name_lookup(iata_code):
    return af[af['iata_code'] == iata_code].iloc[0]['name']
def iata_to_coords(iata_code):
    return af[af['iata_code'] == iata_code].iloc[0]['coordinates']

df['origin'] = df['origin'].apply(iata_to_name_lookup)
df['destination_coords'] = df['destination'].apply(iata_to_coords)
df['destination'] = df['destination'].apply(iata_to_name_lookup)

In [20]:
df = gpd.GeoDataFrame(df)
df['lat'] = df['destination_coords'].str.split(',').str[1]
df['long'] = df['destination_coords'].str.split(',').str[0]
df['geometry'] = gpd.points_from_xy(df['long'], df['lat'], crs='epsg:4326')

US_outline_frame = gpd.read_file('gz_2010_us_outline_20m').to_crs('epsg:4326')
def internation_flight_check(x):
    return US_outline_frame.contains(x)
df['In USA'] = df['geometry'].apply(internation_flight_check)

In [21]:
import folium
m = folium.Map(location=(41.881832, -87.623177), zoom_start=5, min_zoom = 3, max_bounds=True)

for index, info in df.iterrows():
    
    destination_Lat = info['lat']
    destination_Long = info['long']
    destination = info['destination']
    price = info['price']
    tooltip = f'{destination}<br>$ {price}'

    
    folium.Marker([destination_Lat, destination_Long], tooltip=tooltip ).add_to(m)
    
m